In [2]:
from typing import Tuple, List
from PIL import Image, ImageDraw


# TextBox class
class TextBox:
    def __init__(self, text: str, color: Tuple[int, int, int], coordinates: Tuple[int, int]):
        self.color = color
        self.coordinates = coordinates
        self.text = text

    def draw(self, draw_obj: ImageDraw.Draw):
        draw_obj.text(self.coordinates, self.text, fill=self.color)


# Rectangle class
class Rectangle:
    def __init__(self, width: int, height: int, color: Tuple[int, int, int], coordinates: Tuple[int, int]):
        self.color = color
        self.coordinates = coordinates
        self.width = width
        self.height = height

    def draw(self, draw_obj: ImageDraw.Draw):
        x, y = self.coordinates
        draw_obj.rectangle([x, y, x + self.width, y + self.height], fill=self.color)


# Arrow class
class Arrow:
    def __init__(self, start_coordinates: Tuple[int, int], end_coordinates: Tuple[int, int], color: Tuple[int, int, int]):
        self.color = color
        self.start_coordinates = start_coordinates
        self.end_coordinates = end_coordinates

    def draw(self, draw_obj: ImageDraw.Draw):
        draw_obj.line([self.start_coordinates, self.end_coordinates], fill=self.color, width=2)


# Polygon class
class Polygon:
    def __init__(self, vertices: list, color: Tuple[int, int, int]):
        self.color = color
        self.vertices = vertices

    def draw(self, draw_obj: ImageDraw.Draw):
        draw_obj.polygon(self.vertices, fill=self.color)


# Ellipse class
class Ellipse:
    def __init__(self, width: int, height: int, color: Tuple[int, int, int], coordinates: Tuple[int, int]):
        self.color = color
        self.coordinates = coordinates
        self.width = width
        self.height = height

    def draw(self, draw_obj: ImageDraw.Draw):
        x, y = self.coordinates
        draw_obj.ellipse([x, y, x + self.width, y + self.height], fill=self.color)


# Canvas class
class Canvas:
    def __init__(self, width: int, height: int):
        self.shapes = {}  # Dictionary to store shapes, {id: shape_object}
        self.width = width
        self.height = height
        self.image = Image.new("RGB", (width, height), "white")

    def add(self, shape_id: str, shape):
        if shape_id in self.shapes:
            raise ValueError(f"Shape ID '{shape_id}' already exists on the canvas")
        self.shapes[shape_id] = shape

    def remove(self, shape_id: str):
        if shape_id not in self.shapes:
            raise ValueError(f"Shape ID '{shape_id}' does not exist on the canvas")
        del self.shapes[shape_id]

    def draw(self):
        draw_obj = ImageDraw.Draw(self.image)
        for shape in self.shapes.values():
            shape.draw(draw_obj)
        self.image.show()


# Superclass: Element
class Element:
    def __init__(self, coordinates: Tuple[int, int], color: Tuple[int, int, int]):
        self.coordinates = coordinates
        self.color = color

    def draw(self, draw_obj: ImageDraw.Draw):
        raise NotImplementedError("Subclasses must implement the 'draw' method.")


# Subclass: TextBox
class TextBox(Element):
    def __init__(self, text: str, coordinates: Tuple[int, int], color: Tuple[int, int, int]):
        super().__init__(coordinates, color)
        self.text = text

    def draw(self, draw_obj: ImageDraw.Draw):
        draw_obj.text(self.coordinates, self.text, fill=self.color)


# Subclass: Rectangle
class Rectangle(Element):
    def __init__(self, width: int, height: int, coordinates: Tuple[int, int], color: Tuple[int, int, int]):
        super().__init__(coordinates, color)
        self.width = width
        self.height = height

    def draw(self, draw_obj: ImageDraw.Draw):
        x, y = self.coordinates
        draw_obj.rectangle([x, y, x + self.width, y + self.height], fill=self.color)


# Subclass: Arrow
class Arrow(Element):
    def __init__(self, start_coordinates: Tuple[int, int], end_coordinates: Tuple[int, int], color: Tuple[int, int, int]):
        super().__init__(start_coordinates, color)
        self.end_coordinates = end_coordinates

    def draw(self, draw_obj: ImageDraw.Draw):
        draw_obj.line([self.coordinates, self.end_coordinates], fill=self.color, width=2)


# Subclass: Polygon
class Polygon(Element):
    def __init__(self, vertices: List[Tuple[int, int]], color: Tuple[int, int, int]):
        super().__init__(coordinates=(0, 0), color=color)  # No specific origin for polygons
        self.vertices = vertices

    def draw(self, draw_obj: ImageDraw.Draw):
        draw_obj.polygon(self.vertices, fill=self.color)


# Subclass: Ellipse
class Ellipse(Element):
    def __init__(self, width: int, height: int, coordinates: Tuple[int, int], color: Tuple[int, int, int]):
        super().__init__(coordinates, color)
        self.width = width
        self.height = height

    def draw(self, draw_obj: ImageDraw.Draw):
        x, y = self.coordinates
        draw_obj.ellipse([x, y, x + self.width, y + self.height], fill=self.color)


# Subclass: CenteredText (Derived from TextBox)
class CenteredText(TextBox):
    def __init__(self, text: str, center: Tuple[int, int], color: Tuple[int, int, int], font_size: int):
        super().__init__(text, center, color)
        self.font_size = font_size

    def draw(self, draw_obj: ImageDraw.Draw):
        # Calculate text size (requires a font object if advanced sizing is needed)
        text_width, text_height = draw_obj.textsize(self.text)  # Default font
        centered_coordinates = (self.coordinates[0] - text_width // 2, self.coordinates[1] - text_height // 2)
        draw_obj.text(centered_coordinates, self.text, fill=self.color)


# Subclass: TextInEllipse (Combines TextBox and Ellipse)
class TextInEllipse(Ellipse):
    def __init__(self, text: str, width: int, height: int, coordinates: Tuple[int, int], color: Tuple[int, int, int], text_color: Tuple[int, int, int]):
        super().__init__(width, height, coordinates, color)
        self.text = text
        self.text_color = text_color

    def draw(self, draw_obj: ImageDraw.Draw):
        # Draw the ellipse
        super().draw(draw_obj)
        # Calculate text position
        x, y = self.coordinates
        text_width, text_height = draw_obj.textsize(self.text)  # Default font
        text_x = x + (self.width - text_width) // 2
        text_y = y + (self.height - text_height) // 2
        draw_obj.text((text_x, text_y), self.text, fill=self.text_color)


class DashedLine(Element):
    def __init__(self, start_coordinates: Tuple[int, int], end_coordinates: Tuple[int, int], color: Tuple[int, int, int], dash_pattern: List[int], thickness: int):
        super().__init__(start_coordinates, color)
        self.end_coordinates = end_coordinates
        self.dash_pattern = dash_pattern
        self.thickness = thickness

    def draw(self, draw_obj: ImageDraw.Draw):
        x1, y1 = self.coordinates
        x2, y2 = self.end_coordinates
        total_length = ((x2 - x1)**2 + (y2 - y1)**2) ** 0.5
        dash_length = sum(self.dash_pattern)
        dashes = int(total_length // dash_length)

        for i in range(dashes):
            if i % 2 == 0:  # Draw segment for "on" parts of the dash
                segment_start = (x1 + (x2 - x1) * i / dashes, y1 + (y2 - y1) * i / dashes)
                segment_end = (x1 + (x2 - x1) * (i + 1) / dashes, y1 + (y2 - y1) * (i + 1) / dashes)
                draw_obj.line([segment_start, segment_end], fill=self.color, width=self.thickness)


class DegradationDots(Element):
    def __init__(self, coordinates_list: List[Tuple[int, int]], radius: int, color: Tuple[int, int, int]):
        super().__init__(coordinates=(0, 0), color=color)  # No specific origin, uses a list of points
        self.coordinates_list = coordinates_list
        self.radius = radius

    def draw(self, draw_obj: ImageDraw.Draw):
        for coord in self.coordinates_list:
            x, y = coord
            draw_obj.ellipse([x - self.radius, y - self.radius, x + self.radius, y + self.radius], fill=self.color)


class TextInOval(Ellipse):
    def __init__(self, text: str, width: int, height: int, coordinates: Tuple[int, int], color: Tuple[int, int, int], text_color: Tuple[int, int, int]):
        super().__init__(width, height, coordinates, color)
        self.text = text
        self.text_color = text_color

    def draw(self, draw_obj: ImageDraw.Draw):
        # Draw the ellipse
        super().draw(draw_obj)
        # Draw the centered text
        x, y = self.coordinates
        text_width, text_height = draw_obj.textsize(self.text)
        text_x = x + (self.width - text_width) // 2
        text_y = y + (self.height - text_height) // 2
        draw_obj.text((text_x, text_y), self.text, fill=self.text_color)


class LabeledArrow(Element):
    def __init__(self, start_coordinates: Tuple[int, int], end_coordinates: Tuple[int, int], color: Tuple[int, int, int], thickness: int, label: str, label_offset: Tuple[int, int], label_color: Tuple[int, int, int]):
        super().__init__(start_coordinates, color)
        self.end_coordinates = end_coordinates
        self.thickness = thickness
        self.label = label
        self.label_offset = label_offset
        self.label_color = label_color

    def draw(self, draw_obj: ImageDraw.Draw):
        # Draw the arrow line
        draw_obj.line([self.coordinates, self.end_coordinates], fill=self.color, width=self.thickness)
        # Draw the label at specified offset
        label_x = (self.coordinates[0] + self.end_coordinates[0]) // 2 + self.label_offset[0]
        label_y = (self.coordinates[1] + self.end_coordinates[1]) // 2 + self.label_offset[1]
        draw_obj.text((label_x, label_y), self.label, fill=self.label_color)

